In [1]:
import pandas as pd
import numpy as np
import copy
import sys
import os
sys.path.append("../")
from parser.utils import load_json, dfs_cardinality, estimate_scan_in_mb, dumper
from models.feature.single_xgboost_feature import find_top_k_operators, featurize_one_plan, get_top_k_table_by_size
from utils.load_brad_trace import load_trace, create_concurrency_dataset, load_trace_all_version
from models.concurrency.utils import pre_info_train_test_seperation
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from models.single.stage import SingleStage
from models.concurrency.complex_models import ConcurrentRNN
np.set_printoptions(suppress=True)

In [2]:
parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_aurora/aurora_mixed_parsed_queries.json"
#parsed_queries_path = "/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_postgres/postgres_mixed_parsed_queries.json"
plans = load_json(parsed_queries_path, namespace=False)
folder_name = "mixed_redshift"
directory = f"/Users/ziniuw/Desktop/research/Data/AWS_trace/{folder_name}/"
all_raw_trace, all_trace = load_trace_all_version(directory, 8, concat=True)
all_concurrency_df = []
for trace in all_trace:
    concurrency_df = create_concurrency_dataset(trace, engine=None, pre_exec_interval=200)
    all_concurrency_df.append(concurrency_df)
concurrency_df = pd.concat(all_concurrency_df, ignore_index=True)

In [3]:
train_trace_df_sep, eval_trace_df_sep = pre_info_train_test_seperation(concurrency_df)
print(len(train_trace_df_sep), len(eval_trace_df_sep))
np.random.seed(0)
train_idx = np.random.choice(len(concurrency_df), size=int(0.8 * len(concurrency_df)), replace=False)
test_idx = [i for i in range(len(concurrency_df)) if i not in train_idx]
train_trace_df = copy.deepcopy(concurrency_df.iloc[train_idx])
eval_trace_df = concurrency_df.iloc[test_idx]
eval_trace_df = copy.deepcopy(eval_trace_df[eval_trace_df['num_concurrent_queries'] > 0])
print(len(train_trace_df), len(eval_trace_df))

24263 31762
51063 11219


In [4]:
for i, rows in train_trace_df.groupby("query_idx"):
    runtime = rows["runtime"].values
    print(i, len(rows), np.mean(runtime), np.min(runtime), np.max(runtime), np.std(runtime))

0 213 7.216476706831668 3.0992026329040527 14.105448007583618 2.7185046055672
1 198 0.17772019511521464 0.1112494468688964 1.1797688007354736 0.08271800857282727
2 207 4.014342971470045 1.617206335067749 9.45219087600708 1.6644827875275323
3 211 0.35389302692142144 0.2224371433258056 0.7988166809082031 0.09827199059647694
4 212 3.9476491808891296 1.6103508472442627 8.80923318862915 1.6941970689959513
5 218 1.5930830983940614 0.6330249309539795 5.363812208175659 0.7948078413041102
6 208 1.718321170944434 0.8422360420227051 3.667430639266968 0.5840341770344387
7 216 62.575318429205154 34.54681205749512 100.21544647216795 14.249715542395519
8 218 0.14677767141149675 0.0944361686706543 1.1773850917816162 0.08046789188952193
9 218 0.33348455779049374 0.1666772365570068 9.578245639801024 0.6349728026725732
10 215 0.26219703763030294 0.1448307037353515 8.063776016235352 0.537298594953351
11 211 1.282159474223711 0.584784984588623 2.727984666824341 0.510863531072277
12 224 5.20340852865151 2.3

In [4]:
ss = SingleStage(use_table_features=True, true_card=False)
#df = ss.featurize_data(train_trace_df, parsed_queries_path)
df = ss.featurize_data(concurrency_df, parsed_queries_path)
ss.train(df)

Top 20 operators contains 0.9650782102582758 total operators


In [5]:
rnn = ConcurrentRNN(ss, 
                    "redshift",
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_separation=False
                   )
rnn.train(train_trace_df, eval_trace_df, lr=0.001, loss_function="q_loss", val_on_test=True)
#rnn.load_model("checkpoints")

********Epoch 0, training loss: 262.3366677689653 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 88/88 [00:16<00:00,  5.26it/s]


50% absolute error is 0.4374368190765381, q-error is 1.5552023649215698
90% absolute error is 2.156067228317262, q-error is 3.363025856018067
95% absolute error is 3.572301006317142, q-error is 4.434868001937867
********Epoch 5, training loss: 2.105609675183108 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 88/88 [12:43<00:00,  8.67s/it]


50% absolute error is 0.19420850276947021, q-error is 1.28275728225708
90% absolute error is 1.4050401687622076, q-error is 1.9916234254837044
95% absolute error is 2.568176078796388, q-error is 2.3429227590560915
********Epoch 10, training loss: 0.048646608437586544 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 88/88 [00:16<00:00,  5.45it/s]


50% absolute error is 0.18571335077285767, q-error is 1.2448556423187256
90% absolute error is 1.2992297172546392, q-error is 1.9124181509017946
95% absolute error is 2.3307809352874767, q-error is 2.2322344064712523
********Epoch 15, training loss: 0.04770215138568914 || evaluation loss: ********


100%|███████████████████████████████████████████████████████████████████████████| 88/88 [00:16<00:00,  5.38it/s]


50% absolute error is 0.17780086398124695, q-error is 1.23884117603302
90% absolute error is 1.2836357593536378, q-error is 1.861644387245179
95% absolute error is 2.26023578643799, q-error is 2.2161251068115235


KeyboardInterrupt: 

In [6]:
rnn.save_model("checkpoints")

In [7]:
rnn2 = ConcurrentRNN(ss, 
                     "redshift",
                    input_size=len(ss.all_feature[0]) * 2 + 7,
                    embedding_dim=128,
                    hidden_size=256,
                    num_layers=2,
                    loss_function="q_loss",
                    last_output=True,
                    use_separation=False
                   )
rnn2.load_model("checkpoints")

In [9]:
preds, labels = rnn2.predict(eval_trace_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 88/88 [00:17<00:00,  4.98it/s]

50% absolute error is 0.18168844282627106, q-error is 1.2470537424087524
90% absolute error is 1.2665464401245126, q-error is 1.9026330232620239
95% absolute error is 2.31403799057007, q-error is 2.263897228240968


In [61]:
concurrency_df = create_concurrency_dataset(all_trace[0], engine=None, pre_exec_interval=200)
preds, labels = rnn2.predict(concurrency_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 23.03it/s]

50% absolute error is 1.6386100053787231, q-error is 1.2225273251533508
90% absolute error is 14.885701751708984, q-error is 2.4476436853408807
95% absolute error is 23.693497276306143, q-error is 3.6519856929779038


In [58]:
preds2, labels2 = rnn2.predict(eval_trace_df, use_pre_info_only=False)

100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 16.33it/s]

50% absolute error is 2.5743322372436523, q-error is 1.2749279141426086
90% absolute error is 18.791236114501956, q-error is 2.882374620437622
95% absolute error is 28.05649528503415, q-error is 4.55170130729675


In [48]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds2[i])[idx], np.asarray(preds[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

111


array([[  2.8291447,   7.1109896,  79.540054 ],
       [  2.8653407,   2.3412442,   3.416112 ],
       [  2.934177 ,   4.0760264,   4.0271263],
       [  2.9910045,   3.7841144,   3.8717294],
       [  3.0511217,   5.0034904,   3.1730235],
       [  3.148857 ,  18.968218 ,   3.1449356],
       [  3.170916 ,   6.0808353, 106.32712  ],
       [  3.183551 ,   6.5285897,   4.7487164],
       [  3.2096725,  33.221123 ,   2.8220966],
       [  3.2253332,   5.26631  ,   3.3115742],
       [  3.246746 ,   4.4229527,   5.8701944],
       [  3.2892797,   5.2571554,   7.374345 ],
       [  3.2927618,  24.601357 ,   2.9005537],
       [  3.3001895,   3.752159 ,   4.9782295],
       [  3.3236983,   3.4375095,   4.804597 ],
       [  3.3453178,   4.3794765,   3.0326128],
       [  3.4195132,   5.3936315,   8.1688   ],
       [  3.4844189,   6.0924673,   4.3422008],
       [  3.5340056,   4.0386086,   4.554105 ],
       [  3.5757537,   8.585522 ,  32.302185 ],
       [  3.5846162,   6.08425  ,   5.50

In [8]:
i = 210
idx = np.argsort(preds2[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(preds2[i])[idx], np.asarray(labels2[i])[idx]), axis=1)

NameError: name 'preds2' is not defined

In [11]:
i = 2
idx = np.argsort(preds[i])
print(len(idx))
np.stack((np.asarray(preds[i])[idx], np.asarray(labels[i])[idx]), axis=1)

7


array([[  9.086283,  10.558472],
       [  9.326768,  10.257236],
       [ 16.062849,  39.89623 ],
       [ 20.419281,  69.41913 ],
       [ 35.720085,  52.07332 ],
       [ 44.310814, 110.97227 ],
       [ 50.8906  ,  45.67449 ]], dtype=float32)

In [11]:
with open(directory + "postgres_mixed.sql", "r") as f:
    sql = f.readlines()

In [13]:
plans = load_json(parsed_queries_path, namespace=False)

In [16]:
aurora_sql = plans["sql_queries"]

In [15]:
plans.keys()

dict_keys(['parsed_plans', 'parsed_queries', 'sql_queries', 'database_stats', 'run_kwargs', 'skipped', 'blocks_accessed'])

In [21]:
all_index = []
for q in sql:
    i = aurora_sql.index(q.strip())
    all_index.append(i)

In [22]:
all_index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186

In [23]:
sql[-1]

'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;'

In [25]:
sql[206]

'SELECT AVG(person_id), AVG(movie_id) FROM cast_info WHERE id < 25000000;\n'

In [26]:
aurora_sql[207]

'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;'

In [28]:
aurora_sql[-5:]

['SELECT AVG(movie_id), AVG(info_type_id) FROM movie_info WHERE id < 32000000;',
 'SELECT AVG(movie_id), AVG(keyword_id) FROM movie_keyword WHERE id < 42000000;',
 'SELECT AVG(imdb_id) FROM name WHERE id < 42000000;',
 'SELECT AVG(person_id) FROM person_info WHERE id < 42000000;',
 'SELECT AVG(imdb_id), AVG(kind_id), AVG(production_year) FROM title WHERE id < 42000000;']

In [29]:
postgres_plans = copy.deepcopy(plans)

In [30]:
postgres_plans['parsed_plans'] = [plans['parsed_plans'][i] for i in all_index]
postgres_plans['parsed_queries'] = [plans['parsed_queries'][i] for i in all_index]
postgres_plans['sql_queries'] = sql


In [39]:
import json
with open("/Users/ziniuw/Desktop/research/Data/AWS_trace/mixed_postgres/postgres_mixed_parsed_queries.json", "w") as f:
    json.dump(postgres_plans, f)